In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
training_paths = []
interpolate_paths = []
extrapolate_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        p = str(os.path.join(dirname, filename))
        if 'train' in p:
            training_paths.append(p)
        elif 'interpolate' in p:
            interpolate_paths.append(p)
        elif 'extrapolate' in p:
            extrapolate_paths.append(p)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# print(training_paths, '\n')
# print(interpolate_paths, '\n')
# print(extrapolate_paths)

arithmetic_train_paths = []
probability_train_paths = []
for p in training_paths:
    if 'probability' in p:
        probability_train_paths.append(p)
    else:
        arithmetic_train_paths.append(p)
print("TRAINING", '\n')
print(arithmetic_train_paths, '\n')
print(probability_train_paths, '\n')

        
arithmetic_interpolate_paths = []
probability_interpolate_paths = []
for p in interpolate_paths:
    if 'probability' in p:
        probability_interpolate_paths.append(p)
    else:
        arithmetic_interpolate_paths.append(p)
print("INTERPOLATION", '\n')
print(arithmetic_interpolate_paths, '\n')
print(probability_interpolate_paths, '\n')
        
        
arithmetic_extrapolate_paths = []
probability_extrapolate_paths = []
for p in extrapolate_paths:
    if 'probability' in p:
        probability_extrapolate_paths.append(p)
    else:
        arithmetic_extrapolate_paths.append(p)
print("EXTRAPOLATION", '\n')
print(arithmetic_extrapolate_paths, '\n')
print(probability_extrapolate_paths, '\n')

In [ ]:
!pip install numpy==1.15.0
!pip install torch==0.4.1.post2

In [ ]:
import pandas as pd
import nltk
from time import time
import spacy
import gensim
import re
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict

In [ ]:
file1 = open('/kaggle/input/deepmindmath/mathematics_dataset-v1.0/train-hard/probability__swr_p_sequence.txt', 'r')
lines = file1.readlines()
questions = []
answers = []
for lin_num, line in enumerate(lines):
    if lin_num % 2 == 0:
        questions.append(line)
    else:
        answers.append(line)
df = pd.DataFrame()
df['questions'] = pd.Series(questions)[:1000]
df['answers'] = pd.Series(answers)[:1000]

In [ ]:
print(df['questions'].isnull().sum(), df['answers'].isnull().sum())

In [ ]:
!rm -rf *

In [ ]:
!git clone https://github.com/olsenmatthew/LBF.git

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!mv LBF/* ./

In [ ]:
!rm -rf LBF

In [ ]:
import torch
USE_CUDA = torch.cuda.is_available()
USE_CUDA

In [ ]:
from src.train_and_evaluate import *
from src.models import *
import time
import torch.optim
from src.expressions_transfer import *
import sys
import json
import os
import argparse

import warnings
warnings.simplefilter("ignore")

# parser = argparse.ArgumentParser()

# parser.add_argument('--model', default='fix', type=str, choices=['fix','ma-fix','reinforce','mapo'], help='training method')
# parser.add_argument('--nstep', default=50, type=int, help='m-fix')
# parser.add_argument('--name', default='fix', type=str, help='model name')

# options = parser.parse_args()
# model = options.model
# n_step = options.nstep 
# model_name = options.name
model = 'ma-fix'
n_step = 50
model_name = 'ma-fix'

batch_size = 64
embedding_size = 128
hidden_size = 512
n_epochs = 25#0 # TODO configure, original was 100
learning_rate = 1e-3
weight_decay = 1e-5
beam_size = 5
n_layers = 2

data = load_raw_data("data/Math_23K.json")

In [ ]:
data[0]

In [ ]:
dmm = []
for id in range(len(df.index)):
    dmm.append({'id': id, 
                'original_text': df.loc[id]['questions'], 
                'ans': df.loc[id]['answers'].replace('\n', ''),
                'segmented_text': df.loc[id]['questions'],
                'equation': ''
               })
    
#     if id > 1:
#         break

In [ ]:
pairs = transfer_num(dmm)
pairs[0]

In [ ]:
temp_pairs = []
for p in pairs:
    temp_pairs.append((p[0], p[1], p[2], p[3], p[4]))
pairs = temp_pairs
fold_size = int(len(pairs) * 0.2)
fold_pairs = []
for split_fold in range(4):
    fold_start = fold_size * split_fold
    fold_end = fold_size * (split_fold + 1)
    fold_pairs.append(pairs[fold_start:fold_end])
fold_pairs.append(pairs[(fold_size * 4):])

best_acc_fold = []

fold = 1 #we can also iterate all the folds like GTS
pairs_tested = []
pairs_trained = []
for fold_t in range(5):
    if fold_t == fold:
        pairs_tested += fold_pairs[fold_t]
    else:
        pairs_trained += fold_pairs[fold_t]

In [ ]:
input_lang, output_lang, train_pairs, test_pairs = prepare_data(pairs_trained, pairs_tested, 5)
# Initialize models
encoder = EncoderSeq(input_size=input_lang.n_words, embedding_size=embedding_size, hidden_size=hidden_size,
                        n_layers=n_layers)
predict = Prediction(hidden_size=hidden_size, op_nums=5,
                        input_size=2)
generate = GenerateNode(hidden_size=hidden_size, op_nums=5,
                        embedding_size=embedding_size)
merge = Merge(hidden_size=hidden_size, embedding_size=embedding_size)

# predict.load_state_dict(torch.load('./pretrain/predict'))
# encoder.load_state_dict(torch.load('pretrain/encoder'))
# generate.load_state_dict(torch.load('pretrain/generate'))
# merge.load_state_dict(torch.load('pretrain/merge'))
# # the embedding layer is  only for generated number embeddings, operators, and paddings

In [ ]:
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay=weight_decay)
predict_optimizer = torch.optim.Adam(predict.parameters(), lr=learning_rate, weight_decay=weight_decay)
generate_optimizer = torch.optim.Adam(generate.parameters(), lr=learning_rate, weight_decay=weight_decay)
merge_optimizer = torch.optim.Adam(merge.parameters(), lr=learning_rate, weight_decay=weight_decay)

encoder_scheduler = torch.optim.lr_scheduler.StepLR(encoder_optimizer, step_size=20, gamma=0.5)
predict_scheduler = torch.optim.lr_scheduler.StepLR(predict_optimizer, step_size=20, gamma=0.5)
generate_scheduler = torch.optim.lr_scheduler.StepLR(generate_optimizer, step_size=20, gamma=0.5)
merge_scheduler = torch.optim.lr_scheduler.StepLR(merge_optimizer, step_size=20, gamma=0.5)

In [ ]:
# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    predict.cuda()
    generate.cuda()
    merge.cuda()

In [ ]:
buffer_batches = [[] for i in range (len(train_pairs))]
buffer_batches_exp = [[] for i in range (len(train_pairs))]

In [ ]:
stats = {
        'loss': [],
        'test_epoch': [],
        'test_result_acc3': [],
        'test_result_acc1': [],
        'test_result_acc5':[],
        'iteration': []
    }

In [ ]:
iteration = 0
for epoch in range(n_epochs):
    encoder_scheduler.step()
    predict_scheduler.step()
    generate_scheduler.step()
    merge_scheduler.step()
    loss_total = 0
    input_batches, input_lengths, nums_batches, num_pos_batches, num_size_batches, num_ans_batches, num_id_batches = prepare_train_batch(train_pairs, batch_size)
    print("fold:", fold + 1)
    print("epoch:", epoch + 1)
    start = time.time()
    mask_flag = False
    pos = 0
    epo_iteration = 0
    for idx in range(len(input_lengths)): #batch

        if idx < 2 and epoch == 0:
            mask_flag = True
        buffer_batches_train = buffer_batches[pos : pos + len(input_lengths[idx])]
        buffer_batches_train_exp = buffer_batches_exp[pos : pos + len(input_lengths[idx])]

        loss, buffer_batch_new, iterations, buffer_batch_exp = train_tree(
            input_batches[idx], input_lengths[idx], 
            num_size_batches[idx], encoder, predict, generate, merge,
            encoder_optimizer, predict_optimizer, generate_optimizer, merge_optimizer, output_lang, num_pos_batches[idx], num_ans_batches[idx], nums_batches[idx], buffer_batches_train, buffer_batches_train_exp, epoch, model, n_step, mask_flag)
        loss_total += loss
        iteration += iterations
        epo_iteration += iterations
        buffer_batches[pos : pos+len(input_lengths[idx])] = buffer_batch_new
        buffer_batches_exp[pos : pos+len(input_lengths[idx])] = buffer_batch_exp
        pos += len(input_lengths[idx])
    
    loss_total = loss_total if epo_iteration == 0 else loss_total/epo_iteration
    stats['loss'].append(loss_total)
    stats['iteration'].append(iteration)
    print("loss:", loss_total)
    print("training time", time_since(time.time() - start))
    print("--------------------------------")
    if epoch % 1 == 0 or epoch > n_epochs - 5:
        buffer_dict = {
        'id': [],
        'original_text': [],
        'segmented_text': [],
        'gt_equation': [],
        'ans':[],
        'gen_equations': []
        }

        value_ac3 = 0
        eval_total3 = 0
        value_ac1 = 0
        eval_total1 = 0
        value_ac5 = 0
        eval_total5 = 0
        start = time.time()
        for k in range(len(test_pairs)):
            test_batch = test_pairs[k]
            test_exps = []
            test_results = evaluate_tree(test_batch[0], test_batch[1], encoder, predict, generate,
                                        merge, output_lang, test_batch[3], beam_size=beam_size)
            #test_res = test_results[0]
            for i in range (0, len(test_results)):
                test_res = test_results[i]
                val_ac, test_exp = compute_prefix_tree_result(test_res, test_batch[4], output_lang, test_batch[2])

                if val_ac:
                    test_exps.append(test_exp)
                if val_ac:
                    value_ac5 += 1
                eval_total5 += 1

                if i < 3:
                    if val_ac:
                        value_ac3 += 1
                    eval_total3 += 1

                if i == 0:
                    if val_ac:
                        value_ac1 += 1
                    eval_total1 += 1

            # id2 = int(test_pairs[k][7])
            # buffer_dict['id'].append(id2)
            # id2 = id2 - 1
            # buffer_dict['original_text'].append(data[id2]['original_text'])
            # buffer_dict['segmented_text'].append(data[id2]['segmented_text'])
            # buffer_dict['ans'].append(data[id2]['ans'])
            # buffer_dict['gt_equation'].append(data[id2]['equation'])
            # buffer_dict['gen_equations'].append(test_exps)

        stats['test_epoch'].append (epoch)
        stats['test_result_acc3'].append(float(value_ac3) / eval_total3)
        stats['test_result_acc1'].append(float(value_ac1) / eval_total1)
        stats['test_result_acc5'].append(float(value_ac5) / eval_total5)

        print(value_ac1, eval_total1)
        print("test_answer_acc5", float(value_ac5) / eval_total5)
        print("test_answer_acc3", float(value_ac3) / eval_total3)
        print("test_answer_acc1", float(value_ac1) / eval_total1)
        print("testing time", time_since(time.time() - start))
        print("------------------------------------------------------")
        torch.save(encoder.state_dict(), "models/encoder")
        torch.save(predict.state_dict(), "models/predict")
        torch.save(generate.state_dict(), "models/generate")
        torch.save(merge.state_dict(), "models/merge")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
x_ax = []
y_ax = []
for x,y in enumerate(stats['loss']):
    x_ax.append(x)
    y_ax.append(y)
plt.scatter(x_ax,y_ax)
plt.show()

In [ ]:
for x,y in enumerate(stats['test_result_acc1']):
    x_ax.append(x)
    y_ax.append(y)
plt.scatter(x_ax,y_ax)
plt.show()

In [ ]:
for x,y in enumerate(stats['test_result_acc3']):
    x_ax.append(x)
    y_ax.append(y)
plt.scatter(x_ax,y_ax)
plt.show()

In [ ]:
for x,y in enumerate(stats['test_result_acc5']):
    x_ax.append(x)
    y_ax.append(y)
plt.scatter(x_ax,y_ax)
plt.show()